<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit8_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 8: Проксимальный (Ближайший) градиент политики (PPO) с помощью PyTorch 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/thumbnail.png" alt="Unit 8" width="70%"/>


В этом блокноте вы научитесь **кодировать своего агента PPO с нуля с помощью PyTorch, используя реализацию CleanRL в качестве модели**.

Чтобы проверить его надежность, мы собираемся обучить его в среде:

- [LunarLander-v2 🚀](https://www.gymlibrary.dev/environments/box2d/lunar_lander/)


⬇️ Вот пример того, чего вы достигнете. ⬇️

In [1]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/ppo-LunarLander-v2/resolve/main/replay.mp4" type="video/mp4"></video>

## Цели этого блокнота 🏆

В конце записной книжки вы:

- Сможете **закодировать своего агента PPO с нуля с помощью PyTorch**.
- Сможете **отправить своего обученного агента и код в центр** с хорошим видеоповтором и оценочным баллом 🔥.

## Этот блокнот взят из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"  width="70%"/>

## Предварительные условия 🏗️
Прежде чем погрузиться в  блокнот, вам нужно:

🔲  📚 Изучить [PPO, прочитав блок 8](https://huggingface.co/deep-rl-course/unit8/introduction) 🤗  

## Создайте виртуальный дисплей 🔽

Во время записи нам нужно будет сгенерировать видео-повтор. Чтобы сделать это с помощью colab, **нам нужно иметь виртуальный экран, чтобы отображать среду** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyglet==1.5
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()